In [1]:
import pandas as pd
import numpy as np
# import os
from fastai.tabular.all import *
device = torch.device("cpu")


In [2]:

# dtypes = {
# 	'insiderthreat': 'category',
# 	'vector': 'int32',
# 	'date': 'int32',
# 	'user': 'category',
# 	'source': 'category',
# 	'action': 'category'
# }
# df = pd.read_csv("scenario2-training-dataset-transformed-tf2.csv", dtype=dtypes)

negcols=  ['id', 'date', 'user', 'pc', 'type', 'activity']
poscols=  ['id', 'date', 'user', 'pc', 'activity']
positivefiles_path = "/home/haktrak/Public/InsiderThreatDataset/r3.1/logon.csv"
negativefiles_path = "combined_truenegative.csv"

positives = pd.read_csv(positivefiles_path, usecols=poscols, parse_dates=['date'])
negatives = pd.read_csv(negativefiles_path, usecols=negcols, parse_dates=['date'])

# decoder = { 'vector':
# 		{1: 'email',
# 		2: 'device',
# 		0: 'http'}
# 	}

# df.replace(decoder, inplace=True)
# print(df.info())
# print(positives.head())




In [3]:
print(positives.head())
print(negatives.head())

                         id                date     user       pc activity
0  {S7X6-Z2CH44WU-9256VRTB} 2010-01-02 06:34:00  EBH0519  PC-9573    Logon
1  {M5W6-F9WM86XO-3622DJWW} 2010-01-02 06:46:00  LAD0514  PC-2567    Logon
2  {R4Y8-R9YR54PS-5374OMSK} 2010-01-02 06:51:00  YXG0504  PC-4312    Logon
3  {Y1V9-F6VO51FN-3329IZSD} 2010-01-02 07:06:00  JBV0604  PC-3756    Logon
4  {L3N1-D2YO72AX-3581SMQX} 2010-01-02 07:06:00  NLR0174  PC-8272    Logon
                         id                date          user       pc  \
0  {S7A7-Y8QZ65MW-8738SAZP} 2010-01-04 07:12:31  DTAA/RES0962  PC-3736   
1  {G7A8-G1OB94NR-3006NTXH} 2010-01-04 07:35:40  DTAA/BJC0569  PC-2588   
2  {R3L8-N0LW95FR-8358LLXS} 2010-01-04 08:00:38  DTAA/EMZ0196  PC-1479   
3  {I2F1-B5FB51FL-3128HBUL} 2010-01-04 08:02:14  DTAA/ZKH0388  PC-1021   
4  {P7R6-C5TV18CT-1677DWWM} 2010-01-04 08:20:17  DTAA/RES0962  PC-3736   

     type    activity  
0  device     Connect  
1  device     Connect  
2  device     Connect  
3  device

In [4]:
negatives = negatives[negatives.type == 'logon']
negatives.drop('type', axis=1, inplace=True)

positives['threat'] = 1
negatives['threat'] = 0

print(positives.head())
print(negatives.head())

                         id                date     user       pc activity  \
0  {S7X6-Z2CH44WU-9256VRTB} 2010-01-02 06:34:00  EBH0519  PC-9573    Logon   
1  {M5W6-F9WM86XO-3622DJWW} 2010-01-02 06:46:00  LAD0514  PC-2567    Logon   
2  {R4Y8-R9YR54PS-5374OMSK} 2010-01-02 06:51:00  YXG0504  PC-4312    Logon   
3  {Y1V9-F6VO51FN-3329IZSD} 2010-01-02 07:06:00  JBV0604  PC-3756    Logon   
4  {L3N1-D2YO72AX-3581SMQX} 2010-01-02 07:06:00  NLR0174  PC-8272    Logon   

   threat  
0       1  
1       1  
2       1  
3       1  
4       1  
                               id                date          user       pc  \
3517333  {Y6O4-A7KC67IN-0899AOZK} 2010-01-04 00:10:37  DTAA/KEE0997  PC-1914   
3517334  {O5Y6-O7CJ02JC-6704RWBS} 2010-01-04 00:52:16  DTAA/KEE0997  PC-1914   
3517335  {D2D1-C6EB14QJ-2100RSZO} 2010-01-04 01:17:20  DTAA/KEE0997  PC-3363   
3517336  {H9W1-X0MC70BT-6065RPAT} 2010-01-04 01:28:34  DTAA/KEE0997  PC-3363   
3517337  {H3H4-S5AZ00AZ-9560IYHC} 2010-01-04 01:57:30  DTAA

In [5]:
positives.info()
negatives.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 842109 entries, 0 to 842108
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   id        842109 non-null  object        
 1   date      842109 non-null  datetime64[ns]
 2   user      842109 non-null  object        
 3   pc        842109 non-null  object        
 4   activity  842109 non-null  object        
 5   threat    842109 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 38.5+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 849579 entries, 3517333 to 4366911
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   id        849579 non-null  object        
 1   date      849579 non-null  datetime64[ns]
 2   user      849579 non-null  object        
 3   pc        849579 non-null  object        
 4   activity  849579 non-null  object       

In [6]:
# df.groupby(['user', 'insiderthreat'])['source'].count().sort_values(ascending=False)
# df[(df.user == 'KSS1005') & (df.insiderthreat=='1')]
login_df = pd.concat([positives, negatives], axis=0, ignore_index=True)
login_df.shape

(1691688, 6)

In [11]:
login_df['dayofweek'] = login_df['date'].dt.dayofweek
login_df['hour'] = login_df['date'].dt.hour
login_df.drop('date', axis=1, inplace=True)

In [14]:
login_df.drop(['id', 'user', 'pc'], axis=1, inplace=True)

In [15]:
login_df

,activity,threat,dayofweek,hour
0,Logon,1,5,6
1,Logon,1,5,6
2,Logon,1,5,6
3,Logon,1,5,7
4,Logon,1,5,7
...,...,...,...,...
1691683,Logoff,0,5,5
1691684,Logon,0,5,5
1691685,Logoff,0,5,6
1691686,Logoff,0,5,6


In [34]:
dep_var = 'threat'
cat_names = ['activity']
cont_names = ['dayofweek', 'hour']
procs = [Categorify, Normalize]

valid_size = 0.2
NUMBEROFSAMPLES = login_df.shape[0]

np.random.seed(42)
valid_idx = np.random.choice(range(NUMBEROFSAMPLES), int(NUMBEROFSAMPLES*valid_size))

dls = TabularDataLoaders.from_df(login_df, procs=procs, cat_names=cat_names, cont_names=cont_names,
				 y_names=dep_var, valid_idx=valid_idx, bs=64, device=device)
dls.show_batch()

,activity,dayofweek,hour,threat
0,Logon,1.0,7.0,0.0
1,Logon,2.0,9.0,1.0
2,Logoff,3.0,17.0,0.0
3,Logoff,2.0,19.0,1.0
4,Logoff,4.0,16.0,0.0
5,Logoff,4.0,11.0,0.0
6,Logon,2.0,9.0,1.0
7,Logon,3.0,8.0,0.0
8,Logon,4.0,8.0,0.0
9,Logon,2.0,8.0,0.0


In [35]:
learn = tabular_learner(dls, layers=[200,100], metrics=accuracy)


In [41]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.245204,0.245769,0.502263,02:50


In [36]:
learn.show_results()

,activity,dayofweek,hour,threat,threat_pred
0,1.0,-1.398063,0.186395,1.0,0.024561
1,2.0,-0.716825,-0.869868,0.0,-0.017336
2,1.0,-0.035586,1.242658,1.0,0.078459
3,2.0,1.326891,-0.447363,1.0,0.111041
4,2.0,-0.716825,-1.081121,1.0,-0.026037
5,2.0,-1.398063,0.397647,1.0,0.037305
6,1.0,-0.035586,1.242658,1.0,0.078459
7,1.0,-1.398063,1.242658,1.0,0.070974
8,2.0,0.645653,-1.081121,1.0,0.074193


In [42]:
# Predict
row, clas, prob = learn.predict(login_df.iloc[15000])


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [27]:
row

    user  source  action  vector      date  insiderthreat
0  603.0   882.0    32.0     3.0 -1.163744            0.0

In [29]:
from fastai.learner import save_model
learn.save('insiderthreatdetection')

Path('models/insiderthreatdetection.pth')

In [32]:
learn.load('insiderthreatdetection')

In [33]:
row, clas, prob = learn.predict(login_df.iloc[15000])

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [48]:
def combinecsvs(rootpath, threat, list_of_files, outputpath):
	header = True
	mode = 'w'
	# Replace column names
	old_to_new_names = {'url': 'activity'}
	for file in list_of_files:
		chunks = pd.read_csv(rootpath + file, parse_dates=['date'], chunksize=100000)
		print("### Processing file", file, "###")
		for chunk_index, data in enumerate(chunks):
			# @preprocessing
			data['type'] = file.split(".")[0]
			data['threat'] = threat
			if 'content' not in data.columns:
				data['content'] = '-'

			data.rename(old_to_new_names, axis=1, inplace=True)

			data.to_csv(outputpath, header=header, mode=mode, 
				columns=cols, index=False)
			header=False
			mode='a'
			print('Processed Chunk #', chunk_index)

In [45]:

# TRUE POSITIVES
rootpath = "/home/haktrak/Public/InsiderThreatDataset/r3.1/"

list_of_files = ['device.csv', 'http.csv', 'logon.csv', 'email.csv', 'file.csv']
outputpath:str = "combined_truepositive.csv"
threat: bool = True
combinecsvs(rootpath, threat, list_of_files, outputpath)



### Processing file device.csv ###
Processed Chunk # 0
### Processing file http.csv ###
Processed Chunk # 0
Processed Chunk # 1
Processed Chunk # 2
Processed Chunk # 3
Processed Chunk # 4
Processed Chunk # 5
Processed Chunk # 6
Processed Chunk # 7
Processed Chunk # 8
Processed Chunk # 9
Processed Chunk # 10
Processed Chunk # 11
Processed Chunk # 12
Processed Chunk # 13
Processed Chunk # 14
Processed Chunk # 15
Processed Chunk # 16
Processed Chunk # 17
Processed Chunk # 18
Processed Chunk # 19
Processed Chunk # 20
Processed Chunk # 21
Processed Chunk # 22
Processed Chunk # 23
Processed Chunk # 24
Processed Chunk # 25
Processed Chunk # 26
Processed Chunk # 27
Processed Chunk # 28
Processed Chunk # 29
Processed Chunk # 30
Processed Chunk # 31
Processed Chunk # 32
Processed Chunk # 33
Processed Chunk # 34
### Processing file logon.csv ###
Processed Chunk # 0
Processed Chunk # 1
Processed Chunk # 2
Processed Chunk # 3
Processed Chunk # 4
Processed Chunk # 5
Processed Chunk # 6
Processed Chu

In [40]:
# Replace column names
old_to_new_names = {'url': 'activity'}
data.rename(old_to_new_names, axis=1, inplace=True)

# Interpret certain columns as categorical
cat_cols = ['user', 'pc', 'activity']
data[cat_cols] = data[cat_cols].astype('category')
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   id        100000 non-null  object        
 1   date      100000 non-null  datetime64[ns]
 2   user      100000 non-null  category      
 3   pc        100000 non-null  category      
 4   activity  100000 non-null  category      
dtypes: category(3), datetime64[ns](1), object(1)
memory usage: 2.8+ MB
